In [1]:
from os.path import join as pjoin
import numpy as np
import pandas as pd
from datetime import datetime 
from pytz import timezone
import pytz
import multiprocessing
#import pendulum

In [2]:
input_filepath = "../data/raw"
train_df = pd.read_csv(input_filepath + "/train.csv")
building_df = pd.read_csv(input_filepath + "/building_metadata.csv")
train_df = train_df.merge(building_df, on = "building_id", how = "left")

In [3]:
site_info = pd.read_csv('../data/site_info.csv', delimiter = ";")
site_info

,site_id,timezone,country_code,location
0,0,US/Eastern,US,"Orlando, FL"
1,1,Europe/London,UK,"UK, Southhampton"
2,2,US/Mountain,US,"Tempe, AZ"
3,3,US/Eastern,US,"Washington, WA"
4,4,US/Pacific,US,"San Francisco, CA"
5,5,Europe/London,UK,"UK, London"
6,6,US/Eastern,US,Philadelphia
7,7,Canada/Eastern,CA,Montreal/Ottawa
8,8,US/Eastern,US,"Orlando, FL"
9,9,US/Central,US,"Austin, TX"


In [4]:
# convert the string column into a timezone object
site_info.timezone = site_info.timezone.map(lambda x: pytz.timezone(x))

In [5]:
site_info.timezone[1]

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

In [6]:
train_df.shape

(20216100, 9)

In [7]:
train_df = train_df.merge(site_info, on = "site_id", how = "left")

In [8]:
train_df.shape

(20216100, 12)

In [9]:
def reduce_mem_usage(df, verbose=True):
    """
    Takes an dataframe as argument and adjusts the datatypes of the respective
    columns to reduce memory allocation
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int8).min and
                        c_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (c_min > np.iinfo(np.int16).min and
                      c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min and
                      c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min and
                      c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min and
                        c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min and
                      c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    reduced_mem = 100 * (start_mem - end_mem) / start_mem
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
              .format(end_mem, reduced_mem))
    return df


In [10]:
train_df = reduce_mem_usage(train_df)

Mem. usage decreased to 1233.89 Mb (38.5% reduction)


In [11]:
def adjust_column_types(data_frame):
    """
    Takes a data frame and parses certain columns to the desired type.
    """
    data_frame["timestamp"] = pd.to_datetime(data_frame["timestamp"])
    return data_frame

In [12]:
train_df = adjust_column_types(train_df)

In [13]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL"
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL"
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL"
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL"
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL"


In [14]:
type(train_df)

pandas.core.frame.DataFrame

In [15]:
#short example
ts = train_df['timestamp'][1]
ts

Timestamp('2016-01-01 00:00:00')

In [16]:
tz = train_df['timezone'][1]
tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

In [17]:
# use the timezone's localize object to turn a naive timestamp into a local one
tz.localize(ts)

Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern')

In [18]:
# minimal working example, subset of 100
mwe = train_df.iloc[0:99,]

In [19]:
type(mwe)

pandas.core.frame.DataFrame

In [20]:
mwe_tz = mwe['timezone']

In [21]:
mwe_tz[1]

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

In [22]:
mwe_ts = mwe['timestamp']

In [23]:
# for each combination of timezone and timestamp localize the respective
# timestamp 
ts_local = [tz.localize(ts) for tz, ts in zip(mwe_tz, mwe_ts)]

In [24]:
ts_local[1]

Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern')

In [25]:
mwe.loc[:,"ts_local"] = ts_local

c:\users\mufasa\desktop\uni\master biostatistik\git\ashrae-energy-prediction\env\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\mufasa\desktop\uni\master biostatistik\git\ashrae-energy-prediction\env\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [26]:
mwe.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location,ts_local
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00


In [27]:
# now for all the data
ts = train_df['timestamp']

In [28]:
tz = train_df['timezone']

In [29]:
# this takes aaages 
ts_local = [a.localize(b) for a, b in zip(tz, ts)]

In [30]:
train_df.loc[:,'timestamp_local'] = ts_local

In [31]:
datetime.now()

datetime.datetime(2019, 11, 24, 22, 45, 29, 548106)

In [32]:
train_df.head()b

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location,timestamp_local
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00


In [33]:
train_df.shape

(20216100, 13)

In [34]:
train_df.to_pickle("../data/interim" + "/train_data.pkl")